#**Prepare Data & Package**



In [ ]:
!git clone https://github.com/facebookresearch/DeltaCNN.git
!pip install /content/DeltaCNN

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Preprocessing data: convert to tensors and normalize by subtracting dataset
# mean and dividing by std.
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

# Get data from torchvision.datasets
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

#BEGIN WALGELIJK STUKJE CODE

train_set = []
for j in range(9):
  for i in range(len(train_data)):
    img, ground_truth = train_data[i]
    if (ground_truth == (j+1)):
      train_set.append((img, ground_truth))

test_set = []
for j in range(9):
  for i in range(len(test_data)):
    img, ground_truth = test_data[i]
    if (ground_truth == (j+1)):
      test_set.append((img, ground_truth))

train_data = train_set
test_data = test_set
#EIND WALGELIJK STUKJE CODE

# Define data loaders used to iterate through dataset
train_loader = DataLoader(train_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

#**Import libraries**

In [ ]:
import deltacnn
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchsummary import summary
from PIL import Image
import time

#**Visualize Data**

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(5, 5))
for i in range(25):
    x, _ = train_data[i]
    ax = axs[i // 5][i % 5]
    ax.imshow(x.view(28, 28), cmap='gray')
    ax.axis('off')
    ax.axis('off')
plt.tight_layout()
plt.show()

#**Set-up Experiment**

In [ ]:
#EVAL FUNCTION 
def evaluate_accuracy(data_loader, net, device=torch.device('cuda:0')):
    """Evaluate accuracy of a model on the given data set."""
    net.eval()  #make sure network is in evaluation mode

    acc_sum = torch.tensor([0], dtype=torch.float32, device=device)
    n = 0

    for X, y in data_loader:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        if (y.size(0) != 64):
          return acc_sum.item()/n;
          
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))
            n += y.shape[0] #increases with the number of samples in the batch
    return acc_sum.item()/n

In [ ]:
#NET 
class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_features):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, hidden_channels[0],kernel_size=3,padding=1)
        self.relu1 = nn.ReLU()
        self.max_pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(hidden_channels[0], hidden_channels[1],kernel_size=5,padding=2)
        self.relu2 = nn.ReLU()
        self.max_pool2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(7*7*hidden_channels[1], out_features)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.max_pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x) 
        return x

In [ ]:
#DELTACNN NET
import deltacnn
class DeltaCNNNet(deltacnn.DCModule):
    def __init__(self, in_channels, hidden_channels, out_features):
        super(DeltaCNNNet, self).__init__()
        self.sparsify = deltacnn.DCSparsify()
        self.conv1 = deltacnn.DCConv2d(in_channels,hidden_channels[0], kernel_size=3, padding=1, activation="relu", dense_out=False)
        self.max_pool1 = deltacnn.DCMaxPooling(kernel_size=2, stride=2, padding=0, dilation=1)
        self.conv2 = deltacnn.DCConv2d(hidden_channels[0], hidden_channels[1], kernel_size=5, padding=2, activation="relu", dense_out=False)
        self.max_pool2 = deltacnn.DCMaxPooling(kernel_size=2, stride=2, padding=0, dilation=1)
        self.densify = deltacnn.DCDensify()
        self.fc = nn.Linear(7*7*hidden_channels[1], out_features)

    def forward(self, x):
        x = x.cuda()
        x = self.sparsify(x)
        x = self.conv1(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.max_pool2(x)
        x = self.densify(x)
        x = x.reshape(x.size(0), (x.size(1)*x.size(2)*x.size(3)))
        x = self.fc(x)
        return x

#**Define parameters**

In [ ]:
n_runs = 25

in_channels = 1 
hidden_channels = [5, 6]
out_features = 10 
device = torch.device('cuda:0')
learning_rate = 0.001
epochs = 6

#**Experiment**

In [ ]:
time_lst_regular = []
train_loss_regular = []
train_acc_regular = []
test_acc_regular = []

for r in range(n_runs):
  t1 = time.perf_counter()
  ###CODE
  print(r)
  train_losses = []
  train_accs = []
  test_accs = []
  net = Net(in_channels, hidden_channels, out_features)
  optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate)
  criterion = nn.CrossEntropyLoss()
  for epoch in range(epochs):
      net.train()
      net.to(device)
      for i, (x_batch, y_batch) in enumerate(train_loader):
          if (y_batch.size(0) != 64):
            break
          x_batch, y_batch = x_batch.to(device), y_batch.to(device)
          optimizer.zero_grad()
          y_pred = net(x_batch)
          loss = criterion(y_pred, y_batch)
          train_losses.append(loss)          
          loss.backward()
          optimizer.step()

      train_acc = 100*evaluate_accuracy(train_loader, net.to(device))
      test_acc = 100*evaluate_accuracy(test_loader, net.to(device))      
      train_accs.append(train_acc)
      test_accs.append(test_acc)

      # # Print performance
      # print('Epoch: {:.0f}'.format(epoch+1))
      # print('Accuracy of train set: {:.00f}%'.format(train_acc))
      # print('Accuracy of test set: {:.00f}%'.format(test_acc))
      # print('')
  
  train_loss_regular.append(train_losses[-1])
  train_acc_regular.append(train_accs[-1])
  test_acc_regular.append(test_accs[-1])

  ###CODE
  t2 = time.perf_counter()
  time_run = t2-t1
  time_lst_regular.append(time_run)

print(time_lst_regular)
print(train_loss_regular)
print(train_acc_regular)
print(test_acc_regular)

In [ ]:
time_lst_DELTA = []
train_loss_DELTA = []
train_acc_DELTA = []
test_acc_DELTA = []

for r in range(n_runs):
  t1 = time.perf_counter()
  ###CODE
  print(r)
  train_losses = []
  train_accs = []
  test_accs = []

  net = DeltaCNNNet(in_channels, hidden_channels, out_features)
  net.to(device, memory_format=torch.channels_last)
  net.process_filters()
  optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate)
  criterion = nn.CrossEntropyLoss()

  for epoch in range(epochs):
      net.train()
      net.to(device)
      net = net.cuda()
      for i, (x_batch, y_batch) in enumerate(train_loader):
          if (y_batch.size(0) != 64):
            break
          x_batch, y_batch = x_batch.to(device), y_batch.to(device)
          optimizer.zero_grad()
          y_pred = net(x_batch)
          loss = criterion(y_pred, y_batch)
          train_losses.append(loss)          
          loss.backward()
          optimizer.step()

      train_acc = 100*evaluate_accuracy(train_loader, net.to(device))
      test_acc = 100*evaluate_accuracy(test_loader, net.to(device))      
      train_accs.append(train_acc)
      test_accs.append(test_acc)

      # # Print performance
      # print('Epoch: {:.0f}'.format(epoch+1))
      # print('Accuracy of train set: {:.00f}%'.format(train_acc))
      # print('Accuracy of test set: {:.00f}%'.format(test_acc))
      # print('')
  
  train_loss_DELTA.append(train_losses[-1])
  train_acc_DELTA.append(train_accs[-1])
  test_acc_DELTA.append(test_accs[-1])

  ###CODE
  t2 = time.perf_counter()
  time_run = t2-t1
  time_lst_DELTA.append(time_run)

print(time_lst_DELTA)
print(train_loss_DELTA)
print(train_acc_DELTA)
print(test_acc_DELTA)

In [ ]:
plt.scatter(time_lst_regular, test_acc_regular, label='Regular CNN')
plt.scatter(time_lst_DELTA, test_acc_DELTA, label= 'Delta CNN')
plt.legend()
plt.grid()
plt.xlabel('Time per run (s)')
plt.ylabel('Test accuracy per run (%)')

plt.show()

In [ ]:
#EVAL FUNCTION 
def evaluate_accuracy(data_loader, net, device=torch.device('cuda:0'), b_size=64):
    """Evaluate accuracy of a model on the given data set."""
    net.eval()  #make sure network is in evaluation mode

    acc_sum = torch.tensor([0], dtype=torch.float32, device=device)
    n = 0

    for X, y in data_loader:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        if (y.size(0) != b_size):
          return acc_sum.item()/n;
          
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))
            n += y.shape[0] #increases with the number of samples in the batch
    return acc_sum.item()/n

n_runs = 1

in_channels = 1 
hidden_channels = [5, 6]
out_features = 10 
device = torch.device('cuda:0')
learning_rate = 0.001
epochs = 6

batch_lst = [64, 64, 64, 512, 1048]

b_time_lst_regular = []
b_train_loss_regular = []
b_train_acc_regular = []
b_test_acc_regular = []

for b_size in batch_lst:
  train_loader = DataLoader(train_data, batch_size=b_size, shuffle=False)
  test_loader = DataLoader(test_data, batch_size=b_size, shuffle=False)

  time_lst_regular = []
  train_loss_regular = []
  train_acc_regular = []
  test_acc_regular = []

  for r in range(n_runs):
    t1 = time.perf_counter()
    ###CODE
    print(r)
    train_losses = []
    train_accs = []
    test_accs = []
    net = Net(in_channels, hidden_channels, out_features)
    optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        net.train()
        net.to(device)
        for i, (x_batch, y_batch) in enumerate(train_loader):
            if (y_batch.size(0) != b_size):
              break
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = net(x_batch)
            loss = criterion(y_pred, y_batch)
            train_losses.append(loss)          
            loss.backward()
            optimizer.step()

        train_acc = 100*evaluate_accuracy(train_loader, net.to(device), b_size)
        test_acc = 100*evaluate_accuracy(test_loader, net.to(device), b_size)      
        train_accs.append(train_acc)
        test_accs.append(test_acc)

        # # Print performance
        # print('Epoch: {:.0f}'.format(epoch+1))
        # print('Accuracy of train set: {:.00f}%'.format(train_acc))
        # print('Accuracy of test set: {:.00f}%'.format(test_acc))
        # print('')
    
    train_loss_regular.append(train_losses[-1])
    train_acc_regular.append(train_accs[-1])
    test_acc_regular.append(test_accs[-1])

    ###CODE
    t2 = time.perf_counter()
    time_run = t2-t1
    time_lst_regular.append(time_run)

  print(time_lst_regular)
  print(train_loss_regular)
  print(train_acc_regular)
  print(test_acc_regular)

  b_time_lst_regular.append(time_lst_regular[0])
  b_train_loss_regular.append(train_loss_regular[0])
  b_train_acc_regular.append(train_acc_regular[0])
  b_test_acc_regular.append(test_acc_regular[0])

print(b_time_lst_regular)
print(b_train_loss_regular)
print(b_train_acc_regular)
print(b_test_acc_regular)
  
  

